# # Introduction to Modeling Libraries 

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## 13.1 Interfacing Between pandas and Model Code

In [2]:
#模型开发的通常工作流是使用pandas进行数据加载和清洗，然后切换到建模库进行建模
#将dataframe转换为numpy数组
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
data.values  #返回结果类型是numpy数组

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
df2 = pd.DataFrame(data.values,columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [6]:
#使用.values属性时，最好全部是数值类型
df3 = data.copy()
df3['strings'] = ['a','b','c','d','e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [7]:
df3.values   #返回的结果是python对象的ndarray

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [8]:
model_cols = ['x0','x1']
data.loc[:,model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [9]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [10]:
#替换category列为哑变量
dummies = pd.get_dummies(data.category,prefix='category')

In [11]:
dummies

,category_a,category_b
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [13]:
data_with_dummies = data.drop('category',axis=1).join(dummies)

In [14]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 13.2 Creating Model Descriptions with Patsy

In [15]:
#Patsy是python的一个库，使用简短的字符串"公式语法"描述统计模型(尤其是线性模型)

y ~ x0 + x1

In [16]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [17]:
#patsy.dmatrices函数接收一个公式字符串和一个数据集(可以是dataframe或数组的字典)

In [18]:
import patsy

In [19]:
y,x = patsy.dmatrices('y ~ x0 + x1',data)

In [20]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [21]:
x

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [22]:
#这些Patsy的DesignMatrix实例是numpy的ndarray,带有附加元数据
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [23]:
np.asarray(x)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [26]:
#你可能会想intercept是哪里来的，这是线性模型(比如最小二乘法)的惯例用法。
#添加 +0 到模型可以不显示intercept
patsy.dmatrices('y ~ x0 + x1 + 0',data)

(DesignMatrix with shape (5, 1)
      y
   -1.5
    0.0
    3.6
    1.3
   -2.0
   Terms:
     'y' (column 0),
 DesignMatrix with shape (5, 2)
   x0     x1
    1   0.01
    2  -0.01
    3   0.25
    4  -4.10
    5   0.00
   Terms:
     'x0' (column 0)
     'x1' (column 1))

In [27]:
patsy.dmatrices('y ~ x0 + x1 + 0',data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [28]:
#patsy对象可以直接传递到算法(比如numpy.linalg.lstsq)中执行普通最小二乘回归
coef, resid , _ ,_ = np.linalg.lstsq(x,y)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [29]:
coef

array([[ 0.3129],
       [-0.0791],
       [-0.2655]])

In [30]:
resid

array([19.6379])

In [32]:
coef = pd.Series(coef.squeeze(),index=x.design_info.column_names)

In [33]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

In [35]:
#designmatrix对象的元数据保留在design_info属性中
x.design_info

DesignInfo(['Intercept', 'x0', 'x1'],
           factor_infos={EvalFactor('x0'): FactorInfo(factor=EvalFactor('x0'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1),
                         EvalFactor('x1'): FactorInfo(factor=EvalFactor('x1'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1)},
           term_codings=OrderedDict([(Term([]),
                                      [SubtermInfo(factors=(),
                                                   contrast_matrices={},
                                                   num_columns=1)]),
                                     (Term([EvalFactor('x0')]),
                                      [SubtermInfo(factors=(EvalFactor('x0'),),
                                                   contrast_matrices={},
     

In [36]:
x.design_info.column_names

['Intercept', 'x0', 'x1']

### Data Transformations in Patsy Formulas

In [39]:
#可以将python代码与patsy公式结合
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [40]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [41]:
#常见的变量转换包括标准化(平均值为0，方差为1)和去中心化(减去平均值)
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [42]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [53]:
hh = np.asarray(X)[:,1]

In [54]:
hh.mean()

0.0

In [55]:
hh.std()

0.9999999999999999

In [56]:
#建模的其中一步是用新的数据集去评估模型，这时候用新数据有小心，
#因为必须转换成标准化的数据和中心化
#patsy.build_design_matrices函数可以应用于转换新数据，使用原始样本数据集的保存信息
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})

In [57]:
new_X = patsy.build_design_matrices([X.design_info],new_data)

In [58]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [59]:
#patsy中的加号不是加法意义，当你按照名称将数据集的列相加时，必须用特殊函数I将它们封装其俩
y,X = patsy.dmatrices('y ~ I(x0 + x1)',data)

In [60]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [62]:
#patsy库的builtins模块有些内置的转换，参见文档

### Categorical Data and Patsy

In [63]:
#非数值数据可以用多种方式转换为模型设计矩阵
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

In [64]:
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [65]:
y,X = patsy.dmatrices('v2 ~ key1',data)

In [66]:
#在patsy公式中使用非数值型数据，它们会默认转换为虚拟变量
X  

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [67]:
#从模型中忽略截距,每个分类值的列都会包括在设计矩阵的模型中
y,X = patsy.dmatrices('v1 ~ key1 + 0',data)

In [68]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [70]:
#使用C函数，数值列可以截取为分类量
y,X = patsy.dmatrices('v2 ~ C(key2)',data)

In [71]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [72]:
#当你在模型中使用多个分类名
data['key2'] = data['key2'].map({0:'zero',1:'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [73]:
y,X = patsy.dmatrices('v2 ~ key1 + key2',data)

In [74]:
X
#返回的结果b是1，zero是1

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [75]:
#key1:key2形式的相交部分，相当于and
y,X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2',data)

In [76]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## 13.3 Introduction to statsmodels¶

In [77]:
#statsmodels是python进行拟合多种统计模型，进行统计试验和数据探索可视化的库
#但是statsmodels中没有贝叶斯方法和机器学习
#statesmdels中包含的模型有：线性模型，线性混合效应模型，时间序列过程和状态空间模型，广义矩估计

### Estimating Linear Models

In [78]:
#statsmodels的线性模型有两种不同的接口:基于数组，和基于公式
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [97]:
#从一些随机数据生成一个线性模型
def dnorm(mean,variance,size=1):
    if isinstance(size,int):
        size = size,    #注意加,
    return mean + np.sqrt(variance) * np.random.randn(*size)

#for reproducibility
np.random.seed(12345)
N = 100
#np.c_按行连接两个矩阵，要求矩阵的行相等，np.r_按列连接两个矩阵
X = np.c_[dnorm(0,0.4,size=N),
          dnorm(0,0.6,size=N),
          dnorm(0,0.2,size=N)]
eps = dnorm(0,0.1,size=N)
beta = [0.1,0.3,0.5]
y = np.dot(X,beta) + eps

In [92]:
# f = dnorm(0,0.1,size=N)

In [94]:
# f.shape

(100,)

In [87]:
# def dnormf(size=1):
#     if isinstance(size,int):
#         size = size,
#     return np.random.randn(*size)

In [95]:
# dnormf(size=5)

In [100]:
# d= np.array([7,8,9])
# e=np.array([1, 2, 3])
# np.c_[d,e]

array([[7, 1],
       [8, 2],
       [9, 3]])

In [101]:
X[:5]

array([[-0.1295, -1.2128,  0.5042],
       [ 0.3029, -0.4357, -0.2542],
       [-0.3285, -0.0253,  0.1384],
       [-0.3515, -0.7196, -0.2582],
       [ 1.2433, -0.3738, -0.5226]])

In [102]:
np.dot(X,beta)   #因为beta是(3,) X是(100*3)  是X左乘beta

array([-0.1247, -0.2275,  0.0287, -0.3801, -0.2491, -0.1048, -0.3309,
        0.2116, -0.2565,  0.4167,  0.2667,  0.1605,  0.1231, -0.0729,
       -0.1824,  0.2822, -0.2647,  0.0664,  0.5436, -0.4681, -0.6573,
       -0.1363,  0.0726, -0.3094,  0.0395,  0.2042,  0.2918, -0.0875,
       -0.1751,  0.1298, -0.5297, -0.8605, -0.0053,  0.4334, -0.0769,
       -0.3768, -0.014 , -0.0417, -0.1371,  0.1516, -0.6465,  0.2875,
        0.2308, -0.3569,  0.1934, -0.3151,  0.0566, -0.1045,  0.3226,
        0.1589,  0.1402, -0.0583, -0.1145, -0.5211,  0.2773, -0.6924,
       -0.2413,  0.1144,  0.0496,  0.0045, -0.2815,  0.1521, -0.4606,
        0.9726, -0.325 ,  0.0555, -0.0869,  0.2476,  0.3156,  0.197 ,
        0.522 , -0.1238, -0.3572, -0.1223, -0.2016, -0.2076, -0.5424,
       -0.4511,  0.0133, -0.2259, -0.1544, -0.2382, -0.1537,  0.2345,
        0.5443,  0.1531, -0.8736,  0.1615, -0.2942,  0.0762, -0.2565,
        0.0456, -0.0874,  0.3164,  0.1829,  0.2116,  0.072 , -0.0739,
        0.2518,  0.2

In [110]:
y[:5]

array([ 0.4279, -0.6735, -0.0909, -0.4895, -0.1289])

In [111]:
#线性模型通常要拟合一个截距. sm.add_constant函数可以添加一个截距的列到现存的矩阵
X_model = sm.add_constant(X)

In [113]:
X_model[:5]

array([[ 1.    , -0.1295, -1.2128,  0.5042],
       [ 1.    ,  0.3029, -0.4357, -0.2542],
       [ 1.    , -0.3285, -0.0253,  0.1384],
       [ 1.    , -0.3515, -0.7196, -0.2582],
       [ 1.    ,  1.2433, -0.3738, -0.5226]])

In [114]:
#sm.OLS类可以拟合一个普通最小二乘回归
model = sm.OLS(y,X)

In [115]:
#这个模型的fit方法返回一个回归结果的对象，它包含估计的模型参数和其他内容
results = model.fit()

In [117]:
results.params

array([0.1783, 0.223 , 0.501 ])

In [120]:
results.summary()  #注意结果是没有截距项的

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Mon, 05 Aug 2019   Prob (F-statistic):                    7.44e-12
Time:                        17:26:51   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.001       0.073       0.283
x2             0.2230      0.046      4.818      0.000       0.131       0.315
x3             0.5010      0.080      6.237      0.000       0.342       0.660
==============================================================================
Omnibus:                        4.662   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.097   Jarque-Bera (JB):                4.098
Skew:                           0.481   Prob(JB):                        0.129
Kurtosis:                       3.243   Cond. No.                         1.74
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [121]:
#假设所有模型的数据都在一个dataframe里
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data

,col0,col1,col2
0,-0.129468,-1.212753,0.504225
1,0.302910,-0.435742,-0.254180
2,-0.328522,-0.025302,0.138351
3,-0.351475,-0.719605,-0.258215
4,1.243269,-0.373799,-0.522629
5,0.881267,-0.028090,-0.368960
6,0.058760,0.848485,-1.182616
7,0.178192,0.759824,-0.068417
8,0.486373,-0.456615,-0.336269
9,0.788315,1.225180,-0.059305


In [122]:
data['y'] = y

In [123]:
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [124]:
#使用statsmodels的公式API和patsy的公式字符串
results = smf.ols('y ~ col0 + col1 +col2',data).fit()

In [125]:
results.summary()  #结果是带有截距项的
#当使用公式API和字符串对象项时，不需要使用add_constant

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     24.68
Date:                Mon, 05 Aug 2019   Prob (F-statistic):           6.37e-12
Time:                        17:31:37   Log-Likelihood:                -33.835
No. Observations:                 100   AIC:                             75.67
Df Residuals:                      96   BIC:                             86.09
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0336      0.035      0.952      0.343      -0.036       0.104
col0           0.1761      0.053      3.320      0.001       0.071       0.281
col1           0.2248      0.046      4.851      0.000       0.133       0.317
col2           0.5148      0.082      6.304      0.000       0.353       0.677
==============================================================================
Omnibus:                        4.504   Durbin-Watson:                   2.223
Prob(Omnibus):                  0.105   Jarque-Bera (JB):                3.957
Skew:                           0.475   Prob(JB):                        0.138
Kurtosis:                       3.222   Cond. No.                         2.38
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [126]:
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [127]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

In [128]:
# model_inte= sm.OLS(y,X_model)

In [131]:
# model_inte.fit().params  #结果带有截距项的

In [132]:
#给出一个样本外的数据，可以估计模型的预测值
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

### Estimating Time Series Processes

In [134]:
init_x = 4

import random
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

$y_t = b_0y_{t-1} + b_1y_{t-2} + \epsilon_t$

In [137]:
len(values)

1002

In [138]:
#这个AR(2)结构有两个参数0.8和-0.4
#如果你不知道滞后的个数，可以用较多的滞后量来拟合这个模型
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

In [141]:
results.params

array([ 0.0079,  0.7107, -0.3169, -0.0672,  0.0041, -0.0008])

In [144]:
results2 = model.fit(2)

In [145]:
results2.params  #结果中第一个参数是截距，后两个是b0，b1的估计值

array([ 0.0078,  0.7468, -0.3562])

## 13.4 Introduction to scikit-learn

In [146]:
train = pd.read_csv('datasets/titanic/train.csv')

In [147]:
test = pd.read_csv('datasets/titanic/test.csv')

In [150]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [149]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [152]:
#statsmodels和scikit-learn通常不能接收缺失数据，因此需要自行查找是否包含缺失值
train.isnull().sum()
#从结果看age列有缺失值177个，cabin列有缺失值687个

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [153]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [154]:
#我想用年龄作为预测值，但是它存在缺失值，先将年龄数据补全
#用训练数据集的中位数补全两个表的空值
impute_value = train['Age'].median()

In [155]:
train['Age'] = train['Age'].fillna(impute_value)

In [156]:
test['Age'] = test['Age'].fillna(impute_value)

In [157]:
#增加一个列IsFemale,作为"Sex"列的编码
train['IsFemale'] = (train['Sex'] == 'female').astype(int)

In [158]:
train[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [159]:
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [160]:
test[:5]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1


In [161]:
#确定模型变量，并创建Numpy数组
predictors = ['Pclass','IsFemale','Age']
X_train = train[predictors].values
X_test = test[predictors].values

In [163]:
y_train = train['Survived'].values

In [165]:
#使用scikit-learn的LogisticRegression模型创建一个模型实例
from sklearn.linear_model import LogisticRegression

In [166]:
model = LogisticRegression()

In [167]:
model.fit(X_train,y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [168]:
y_predict = model.predict(X_test)

In [169]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [170]:
#如果有测试集的真值，则可以将预测值与真实值比较

(y_true == y_predict).mean()

In [172]:
#交叉验证是通过分割训练数据来模拟样本外预测
from sklearn.linear_model import LogisticRegressionCV
model_cv = LogisticRegressionCV(10)
model_cv.fit(X_train,y_train)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [175]:
#要手动进行交叉验证，可以使用cross_val_score函数
from sklearn.model_selection import cross_val_score
model = LogisticRegression(C=10)
scores = cross_val_score(model,X_train,y_train,cv=4)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [176]:
scores

array([0.7723, 0.8027, 0.7703, 0.7883])